In [32]:
# %pip install gensim --user

In [34]:
import sys, os
# sys.path.insert(0, '/datasets/home/51/451/yuz530/.local/lib/python3.7/site-packages/')

In [1]:
!ls .

ABA.cor                 jeff_word2vec.ipynb     test.cor
__pycache__             n2v.cor                 test_python_files.ipynb
data                    n2v.py                  w2v.py


In [2]:
import json
from scipy import sparse
import numpy as np
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
import gensim
from tqdm import tqdm
from node2vec import Node2Vec
import networkx as nx
from itertools import cycle
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction

unable to import 'smart_open.gcs', disabling that module


In [45]:
A = sparse.load_npz('./data/A.npz')
B = sparse.load_npz('./data/B.npz')
P = sparse.load_npz('./data/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap ;D

In [11]:
# ABPBA
def ABPBA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        
# ABA
def ABA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} app_{app_j}'
        
# it = iter_neighbors()

In [12]:
app_i = np.random.choice(np.arange(A.shape[0]))
print(app_i)
api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
print(api_i)
api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
print(api_bi)
if(api_bi > A.shape[1]):
    api_bi = A.shape[1]-1
app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])
app_j

102
12338
699629


73

In [13]:
(B[:, api_i])

<916573x1 sparse matrix of type '<class 'numpy.int64'>'
	with 747 stored elements in Compressed Sparse Row format>

In [14]:
# %lprun -f iter_neighbors a = [next(it) for i in range(10000)] 

In [15]:
#note ValueError: 'a' cannot be empty unless no samples are taken B col all 0 why?

for _ in range(10):
    print(next(ABA()) + '\n')

ValueError: 'a' cannot be empty unless no samples are taken

In [3]:
import n2v
import numpy as np
n2v.node2vec(p=2, q=1, walk_length=20)
# n2v.test()

['app_266',
 'api_69477',
 'api_206438',
 'api_529',
 'api_6293',
 'api_780315',
 'app_381',
 'api_188049',
 'api_427584',
 'app_340',
 'api_529',
 'api_533',
 'app_612',
 'api_473',
 'app_612',
 'api_473',
 'api_1237',
 'api_1391215',
 'api_472',
 'api_306372']

In [4]:
' '.join(n2v.node2vec(p=2, q=1, walk_length=20))

'app_780 api_1499914 api_1500266 api_1499909 api_1500266 app_780 api_1499910 api_403 api_140 api_403 api_369962 api_403 api_369962 api_294324 api_1051 api_3900 api_1051 api_437 api_266561 app_379'

In [69]:
%%time
f = open('n2v.cor', 'w')
for _ in range(10000):
    try:
        f.write(' '.join(node2vec(p=2, q=1, walk_length=20)) + '\n')
    except:
        pass
f.close()

CPU times: user 3h 23min 2s, sys: 9min 43s, total: 3h 32min 45s
Wall time: 1h 22min 46s


In [70]:
!head n2v.cor

app_417 api_184527 app_328 api_241420 app_168 api_184528 api_241412 api_241411 app_414 api_241411 api_184532 api_184534 api_392214 app_49 api_392214 api_1205262 api_392214 api_1204460 api_1204459 api_1205273
app_648 api_1404756 api_1437613 api_1404923 api_1474609 api_1474616 api_1474600 api_1870 api_1404756 api_1870 api_1410542 api_1 api_1410542 api_1 api_1534306 app_909 api_1534302 api_1 api_9072 api_1187199
app_274 api_31276 api_38217 api_1835 api_38217 api_230660 api_116196 app_417 api_230759 app_214 api_230759 app_210 api_230759 api_1780 api_81 api_361800 api_537 api_1263 api_188981 app_15
app_251 api_946212 api_946213 api_946212 api_946209 app_251 api_95413 app_65 api_464224 api_44888 api_44390 app_28 api_302642 api_1195809 api_302642 api_1195809 api_1196722 api_1142 api_1196722 api_1196796
app_580 api_1411839 app_580 api_1411829 app_580 api_1411632 app_580 api_1411957 api_1411958 api_1411808 api_1411958 api_1411808 app_580 api_1412004 api_6000 api_26172 api_595538 api_595875 api_

In [51]:
# f = open('test.cor', 'w')
# for _ in range(10000):
#     f.write(next(it) + '\n')
# f.close()

In [52]:
!head ABA.cor

In [5]:
%%time
import w2v
model = w2v.create_model()

/Users/jeffliao/.local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


CPU times: user 11min 34s, sys: 2min 13s, total: 13min 48s
Wall time: 9min 6s


In [122]:
model

In [6]:
%%time
from gensim import utils
import gensim.models

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'n2v.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)
            yield line.strip().split(' ')

CPU times: user 34 µs, sys: 1e+03 ns, total: 35 µs
Wall time: 37.9 µs


In [72]:
%%time
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

/Users/jeffliao/.local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning:

C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.



CPU times: user 10min 19s, sys: 1min 58s, total: 12min 18s
Wall time: 8min 4s


In [7]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

app_417
api_184527
app_328
api_241420
app_168
api_184528
api_241412
api_241411
app_414
api_184532


In [9]:
model.wv.similarity("app_417", "app_414")

0.26601884

In [10]:
print(model.wv.most_similar(positive=['app_177'], topn=5))

[('app_300', 0.43069976568222046), ('api_106', 0.4216550588607788), ('app_313', 0.41829103231430054), ('api_658', 0.4029352068901062), ('api_423', 0.401669979095459)]


In [11]:
x_vals, y_vals, labels = w2v.reduce_dimensions(model)

w2v.plot_with_plotly(x_vals, y_vals, labels)


In [13]:
from tqdm import tqdm
import numpy as np
from scipy import sparse


class Node2Vec():
    def __init__(self, A_tr, B_tr, P_tr):
        assert 'csr_matrix' in str(type(A_tr))
        self.A_tr_csr = A_tr
        self.A_tr_csc = A_tr.tocsc(copy=True)
        self.B_tr = B_tr
        self.P_tr = P_tr
    
    def get_api_neighbors_A(self, app):
        """Get all API neighbors of an APP from A matrix"""
        assert app.startswith('app_')
        app_id = int(app.split('_')[1])
        neighbor_ids = np.nonzero(self.A_tr_csr[app_id])[1]
        return np.array([f'api_{s}' for s in neighbor_ids])

    def get_app_neighbors_A(self, api):
        """Get all APP neighbors of an API from A matrix"""
        assert api.startswith('api_')
        api_id = int(api.split('_')[1])
        neighbor_ids = np.nonzero(self.A_tr_csc[:, api_id])[0]
        return np.array([f'app_{s}' for s in neighbor_ids])

    def get_api_neighbors_B(self, api):
        """Get all API neighbors of an API from B matrix"""
        assert api.startswith('api_')
        api_id = int(api.split('_')[1])
        neighbor_ids = np.nonzero(self.B_tr[:, api_id])[0]
        ls = [f'api_{s}' for s in neighbor_ids]
        ls.remove(api)
        return np.array(ls)

    def get_api_neighbors_P(self, api):
        """Get all API neighbors of an API from P matrix"""
        assert api.startswith('api_')
        api_id = int(api.split('_')[1])
        neighbor_ids = np.nonzero(self.P_tr[:, api_id])[0]
        ls = [f'api_{s}' for s in neighbor_ids]
        ls.remove(api)
        return np.array(ls)

    def all_neighbors_from_api(self, api):
        """Get all API neighbors of an APP from all matrices (B and P)"""
        assert api.startswith('api_')
        api_id = int(api.split('_')[1])
        nbr_apis = np.concatenate([
            self.get_api_neighbors_B(api),
            self.get_api_neighbors_P(api)
        ])
        nbr_apis = np.unique(nbr_apis)
        nbr_apps = self.get_app_neighbors_A(api)
        # weights later?
        return nbr_apis, nbr_apps
    
    def perform_one_walk_full(self, p=1, q=1, walk_length=20, app=None):
        path = []

        if app is None:
            app = 'app_' + str(np.random.choice(np.arange(self.A_tr_csr.shape[0])))
        prev_nbrs = self.get_api_neighbors_A(app)
        curr_node = np.random.choice(prev_nbrs)
        prev_node = app
        path.append(app)
        path.append(curr_node)

        for i in range(walk_length - 2):
            if curr_node.startswith('api_'):
                nbr_apis, nbr_apps = self.all_neighbors_from_api(curr_node)
                curr_nbrs = np.concatenate([nbr_apis, nbr_apps])
            elif curr_node.startswith('app_'):
                curr_nbrs = self.get_api_neighbors_A(curr_node)
            else: raise AssertionError

            alpha_1 = np.intersect1d(prev_nbrs, curr_nbrs, assume_unique=True)
            alpha_p = prev_node
            alpha_q = np.setdiff1d(
                np.setdiff1d(curr_nbrs, alpha_1, assume_unique=True),
                [alpha_p], assume_unique=True
            )
            alphas = [*alpha_1, alpha_p, *alpha_q]
            assert len(alpha_1) + len(alpha_q) + 1 == len(curr_nbrs)

            probs_q = np.full(len(alpha_q), 1/q/len(alpha_q)) if len(alpha_q) else []
            probs_1 = np.full(len(alpha_1), 1/len(alpha_1)) if len(alpha_1) else []
            probs = [*probs_1, 1/p, *probs_q]
            probs = np.array(probs) / sum(probs)

            new_node = np.random.choice(alphas, p=probs)
            path.append(new_node)
            prev_node = curr_node
            prev_nbrs = curr_nbrs
            curr_node = new_node

        return path
    
    def perform_one_walk_metapath(self, p=1, q=1, walk_length=20, app=None, metapath='APA'):
        path = []

        if metapath == 'APA':
            path_stages = ['A', 'P']

        if app is None:
            app = 'app_' + str(np.random.choice(range(self.A_tr_csr.shape[0])))
        prev_nbrs = self.get_api_neighbors_A(app)
        curr_node = np.random.choice(prev_nbrs)
        prev_node = app
        path.append(app)
        path.append(curr_node)

        prev_stage = 'A'

        for i in range(walk_length - 2):
            stage = path_stages[
                (path_stages.index(prev_stage) + 1) % len(path_stages)
            ]
            print(prev_stage, stage)

            # if curr_node.startswith('api_'):
            #     nbr_apis, nbr_apps = self.all_neighbors_from_api(curr_node)
            #     curr_nbrs = np.concatenate([nbr_apis, nbr_apps])
            # elif curr_node.startswith('app_'):
            #     curr_nbrs = self.get_api_neighbors_A(curr_node)
            # else: raise AssertionError

            if stage.startswith('A'):
                assert curr_node.startswith('app_')
                curr_nbrs = self.get_api_neighbors_A(curr_node)
            elif stage.startswith('B'):
                assert curr_node.startswith('api_')
                nbr_apps = self.get_app_neighbors_A(curr_node)
                nbr_apis = self.get_api_neighbors_B(curr_node)
                curr_nbrs = np.concatenate([nbr_apis, nbr_apps])
            elif stage.startswith('P'):
                assert curr_node.startswith('api_')
                nbr_apps = self.get_app_neighbors_A(curr_node)
                nbr_apis = self.get_api_neighbors_P(curr_node)
                curr_nbrs = np.concatenate([nbr_apis, nbr_apps])
            else: raise AssertionError

            alpha_1 = np.intersect1d(prev_nbrs, curr_nbrs, assume_unique=True)
            alpha_p = prev_node
            alpha_q = np.setdiff1d(
                np.setdiff1d(curr_nbrs, alpha_1, assume_unique=True),
                [alpha_p], assume_unique=True
            )
            alphas = [*alpha_1, *alpha_q, alpha_p]

            # print(len(alpha_1), len(alpha_q), len(curr_nbrs))
            print(prev_node, curr_node)
            # print(np.setdiff1d(alphas, curr_nbrs))
            # print(np.setdiff1d(curr_nbrs, alphas))
            assert len(alphas) == len(curr_nbrs)

            probs_1 = np.full(len(alpha_1), 1/len(alpha_1)) if len(alpha_1) else []
            probs_q = np.full(len(alpha_q), 1/q/len(alpha_q)) if len(alpha_q) else []
            probs = [*probs_1, *probs_q, 1/p]
            probs = np.array(probs) / sum(probs)

            new_node = np.random.choice(alphas, p=probs)
            print(new_node)
            if new_node in alpha_1:
                prev_stage = prev_stage
            elif new_node == alpha_p:
                prev_stage = prev_stage
            elif new_node in alpha_q:
                prev_stage = stage
            else: raise Error('Something went really wrong')

            path.append(new_node)
            prev_node = curr_node
            prev_nbrs = curr_nbrs
            curr_node = new_node

        return path
    
    def perform_walks(self, n, p, q, walk_length):
        # n is how many paths from each app
        n_apps_tr = self.A_tr_csr.shape[0]

        walks = []
        for app_i in tqdm(range(n_apps_tr)):
            app = 'app_' + str(app_i)
            
            for j in range(n):
                path = self.perform_one_walk_full(p, q, walk_length, app=app)
                walks.append(path)

        return walks
    
    def save_corpus(self):
        n=1
        p=2
        q=1
        walk_length=100
        
        outfile = open(f'node2vec_n={n}_p={p}_q={q}_wl={walk_length}.cor', 'w')
        walks = self.perform_walks(n=n, p=p, q=q, walk_length=walk_length)

        print('saving..')
        for walk in tqdm(walks):
            outfile.write(' '.join(walk) + '\n')
        outfile.close()



In [15]:
A = sparse.load_npz('./data/A.npz')
B = sparse.load_npz('./data/B.npz')
P = sparse.load_npz('./data/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap

In [16]:
n = Node2Vec(A, B, P)

In [17]:
n.perform_one_walk_full()

['app_672',
 'api_267',
 'app_672',
 'api_1453972',
 'api_1476474',
 'api_1453924',
 'api_1476474',
 'api_1453987',
 'api_1453994',
 'api_1453987',
 'api_1453902',
 'api_1453987',
 'api_1453902',
 'api_1453985',
 'api_81',
 'api_1300366',
 'api_80',
 'api_1300366',
 'api_1300373',
 'app_439']

In [18]:
n.perform_one_walk_metapath()

A P
app_691 api_885
app_649
P A
api_885 app_649
api_885
P A


AssertionError: 